In [1]:
import os

import pandas as pd
import time
import pickle

In [2]:

from ase.calculators.espresso import Espresso, EspressoProfile
from ase.io.espresso import read_espresso_out

In [3]:
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion
from disalloy.grid_run import run_packet

In [4]:
from ase.io.espresso import read_espresso_out

In [5]:
profile = EspressoProfile(
    command='mpirun -np 2 --allow-run-as-root /qe/q-e/bin/pw.x', 
    pseudo_dir='/qe/q-e/pseudo/SSSP/'
)

pseudopotentials = {
    'Al':'Al.pbe-n-kjpaw_psl.1.0.0.UPF',
    'Co':'Co_pbe_v1.2.uspp.F.UPF',
    'Cr':'cr_pbe_v1.5.uspp.F.UPF',
    'Cu':'Cu.paw.z_11.ld1.psl.v1.0.0-low.upf',
    #'Cu':'Cu.pbe-kjpaw.UPF',
    'Fe':'Fe.pbe-spn-kjpaw_psl.0.2.1.UPF',
    'Mn':'mn_pbe_v1.5.uspp.F.UPF',
    'Ni':'ni_pbe_v1.4.uspp.F.UPF',
    'V':'v_pbe_v1.4.uspp.F.UPF',
     }


input_data = {
    'system': {'ecutwfc': 60, 'ecutrho': 480},
    'disk_io': 'low',  # Automatically put into the 'control' section
    'tstress':True,  # deprecated, put in input_data
    'tprnfor':True,  # deprecated, put in input_data
    'calculation': 'vc-relax',#'scf',
    'conv_thr':1e-8,
    'occupations':'smearing',
    'smearing':'mv',
    'degauss':0.02,
    'mixing_beta':0.3,
    #'diagonalization':'cg',
}

calc_espresso_relax = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    input_data=input_data,
    kspacing=0.5e-1,
)

def parse_calculation_relax(file_name = 'espresso.pwo'):
    with open(file_name, 'r') as f:
        for i in read_espresso_out(f,):
            pass
    return i.copy(), i.get_potential_energy()

def parse_calculation_traj_relax(file_name = 'espresso.pwo'):
    res = []
    with open(file_name, 'r') as f:
        for i in read_espresso_out(f,):
            res.append([i.copy(), i.get_potential_energy(), i.get_forces(), i.get_stress()])
    return res

def relaxer_espresso(row):
    structure = row['init_structure'].copy()
    structure.calc = calc_espresso_relax
    structure.get_potential_energy()
    return parse_calculation_traj_relax()
    
def relaxer_espresso_jitter_ia(row, ia = 0.01):
    structure = row['init_structure'].copy()
    structure = jitter_ia(structure, ia)
    structure.calc = calc_espresso_relax
    structure.get_potential_energy()
    return parse_calculation_traj_relax()
    
def relaxer_espresso_jitter_ic(row, ic = 0.005):
    structure = row['init_structure'].copy()
    structure = jitter_ic(structure, ic)
    # structure.set_cell(structure.cell.array, scale_atoms = True)
    structure.calc = calc_espresso_relax
    structure.get_potential_energy()
    return parse_calculation_traj_relax()

In [6]:
from disalloy.grid_run import run_packet

In [7]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [8]:
df = structure_DB.copy()
# if os.path.exists('df_calc.pkl'):
#     df = pd.read_pickle('df_calc.pkl')

In [9]:
relaxer_dic_espresso = {'calculator_label': 'relaxer_espresso',
                            'relaxer': relaxer_espresso}

In [10]:
print(df.attrs)

{'number_of_structures': {1: 8, 2: 56, 3: 336, 4: 4046, 5: 15904, 6: 17080, 7: 23968, 8: 128540, 9: 140000, 10: 180000}, 'is_exhaustive': {1: True, 2: True, 3: True, 4: True, 5: True, 6: False, 7: False, 8: False, 9: False, 10: False}}


In [11]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 2, 1)
print('{}s'.format(time.time() - ti))

2024-09-05 06:38:27 - starting packet run: attempting 2 runs of size 1

2024-09-05 06:39:22 - index of ran strctures:
 [4, 5]

2024-09-05 06:39:22 - Succesfuly ran packet of 2 runs in 35.53978228569031s

55.452261447906494s


In [12]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 2, 2)
print('{}s'.format(time.time() - ti))

2024-09-05 06:39:28 - starting packet run: attempting 2 runs of size 2

2024-09-05 06:59:52 - index of ran strctures:
 [34, 28]

2024-09-05 06:59:52 - Succesfuly ran packet of 2 runs in 1207.5336208343506s

1224.7591321468353s


In [13]:
df[df[relaxer_dic_espresso['calculator_label']].notnull()]

,init_structure,size,relaxer_espresso
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Fe', [0.0, 0.0, 0.0], index=0)), -447..."
5,"(Atom('Mn', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Mn', [0.0, 0.0, 0.0], index=0)), -287..."
28,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",2,"[[(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom(..."
34,"(Atom('Mn', [0.0, 0.0, 0.0], index=0), Atom('V...",2,"[[(Atom('Mn', [0.0, 0.0, 0.0], index=0), Atom(..."


In [14]:
df.loc[57].init_structure

Atoms(symbols='CuV', pbc=True, cell=[[-1.5, -1.5, 0.0], [1.5, -1.5, 0.0], [0.0, 0.0, 3.0]])

In [15]:
calculated_index = df[df[relaxer_dic_espresso[
    'calculator_label']].notnull()].index

In [18]:
for i, j  in zip(
    df.loc[calculated_index, 'init_structure'], 
    df.loc[calculated_index, 'relaxer_espresso'].array):
    
    print(i.cell.array)
    print(i.cell.cellpar())
    print()
    print(j[-1][0].cell.array)
    print(j[-1][0].cell.cellpar())
    print()
    print(compute_distortion(i, j[-1][0]))
    print()
    print('#'*50)

[[0.79370053 0.79370053 0.        ]
 [0.         0.79370053 0.79370053]
 [0.79370053 0.         0.79370053]]
[ 1.12246205  1.12246205  1.12246205 60.         60.         60.        ]

[[ 1.68474359  1.68474359 -0.        ]
 [-0.          1.68474359  1.68474359]
 [ 1.68474359 -0.          1.68474359]]
[ 2.38258724  2.38258724  2.38258724 60.         60.         60.        ]

1.7281680248448638e-16

##################################################
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[ 2.12132034  2.12132034  2.12132034 60.         60.         60.        ]

[[ 1.75047586  1.75047586  0.        ]
 [-0.          1.75047586  1.75047586]
 [ 1.75047586  0.          1.75047586]]
[ 2.4755467  2.4755467  2.4755467 60.        60.        60.       ]

1.7281680248448638e-16

##################################################
[[-1.5 -1.5  0. ]
 [ 0.  -1.5 -1.5]
 [ 1.5 -3.   1.5]]
[ 2.12132034  2.12132034  3.67423461 73.22134512 73.22134512 60.        ]

[[-1.77037616 -1.77037616  0.      

In [19]:
relaxer_dic_espresso_jitter_ic = {
    'calculator_label': 'relaxer_espresso_jitter_ic',
    'relaxer': relaxer_espresso_jitter_ic}

In [20]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso_jitter_ic, 10, 1,
          copy_calculated=relaxer_dic_espresso['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-05 07:12:01 - starting packet run: attempting 10 runs of size 1

2024-09-05 08:00:44 - index of ran strctures:
 [4, 5]

2024-09-05 08:00:44 - Succesfuly ran packet of 2 runs in 2887.304027557373s

2923.3300359249115s


In [23]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso_jitter_ic, 10, 2,
          copy_calculated=relaxer_dic_espresso['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-05 08:05:48 - starting packet run: attempting 10 runs of size 2



CalledProcessError: Command '['mpirun', '-np', '2', '--allow-run-as-root', '/qe/q-e/bin/pw.x', '-in', 'espresso.pwi']' returned non-zero exit status 1.

In [94]:
relaxer_dic_espresso_jitter_ic = {
    'calculator_label': 'relaxer_espresso_jitter_ic',
    'relaxer': relaxer_espresso_jitter_ic}
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso_jitter_ic, 10, 2,
          copy_calculated=relaxer_dic_espresso['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-05 03:22:23 - starting packet run: attempting 10 runs of size 2

2024-09-05 05:06:14 - index of ran strctures:
 [57, 19, 60]

2024-09-05 05:06:14 - Succesfuly ran packet of 3 runs in 6201.594995498657s

6231.058604955673s


In [95]:
calculated_index = df[df[relaxer_dic_espresso['calculator_label']].notnull()].index

In [102]:
for i, j, k  in zip(df.loc[calculated_index, 'init_structure'], 
                df.loc[calculated_index, 'relaxer_espresso'].array, 
             df.loc[calculated_index, 'relaxer_espresso_jitter_ic'].array):
    print(len(i))
    if len(i) == 1 :
        continue
    print(i.cell.array)
    print(i.cell.cellpar())
    print(j[0].cell.array)
    print(j[0].cell.cellpar())
    print(k[0].cell.array)
    print(k[0].cell.cellpar())
    
    # print(compute_distortion(i, j[0]))
    # print(compute_distortion(i, k[0]))
    print()

1
1
1
2
[[-0.62996052 -0.62996052  0.        ]
 [ 0.         -0.62996052 -0.62996052]
 [ 0.62996052 -1.25992105  0.62996052]]
[ 0.89089872  0.89089872  1.54308184 73.22134512 73.22134512 60.        ]
[[-0.62981427 -0.62981427 -0.        ]
 [ 0.         -0.62981427 -0.62981427]
 [ 0.63039943 -1.2602137   0.63039943]]
[ 0.89069188  0.89069188  1.54367919 73.23203896 73.23203896 60.        ]
[[-0.98602551 -1.3690834   0.35677255]
 [ 0.24998739 -1.47300106 -0.82988477]
 [ 1.58648771 -1.75039278  1.84671916]]
[ 1.7245064   1.70907418  2.99853127 73.65302653 73.24171402 59.99028956]

2
[[-0.62996052 -0.62996052  0.        ]
 [ 0.62996052 -0.62996052  0.        ]
 [ 0.          0.          1.25992105]]
[ 0.89089872  0.89089872  1.25992105 90.         90.         90.        ]
[[-0.60611107 -0.60611107  0.        ]
 [ 0.60611107 -0.60611107  0.        ]
 [ 0.          0.          1.36102334]]
[ 0.85717049  0.85717049  1.36102334 90.         90.         90.        ]
[[-1.2934146  -1.18954308  0.

In [103]:
with open('espresso.pwo', 'r') as f:
    for i in read_espresso_out(f):
        print(i.cell.array)
        print(i.cell.cellpar())
        print()

[[-7.54506491e-01 -7.46576306e-01  6.88145993e-03]
 [ 7.49080296e-01 -7.54419451e-01 -5.21177977e-03]
 [-6.66598308e-04 -5.59178326e-03  1.52051499e+00]]
[ 1.06146294  1.06315434  1.52052542 90.14905201 89.46249156 90.10104192]

[[-0.98570727 -1.22641319  0.28570313]
 [ 0.97508134 -1.23127421  0.26636177]
 [-0.0051407  -0.56586556  2.26565875]]
[ 1.59916679  1.59303747  2.33526012 69.62521701 68.86775473 75.79819062]

[[-1.34427862 -1.4738945  -0.10999482]
 [ 1.33411303 -1.49431395 -0.12084586]
 [ 0.01239988  0.20285602  3.19762008]]
[ 1.99788609  2.00684715  3.20407217 96.00788605 95.98428116 83.95355695]

[[-1.48362422 -1.14610663 -0.36856634]
 [ 1.48075592 -1.1795938  -0.37765433]
 [ 0.01204258  0.72340935  2.90479508]]
[  1.91063926   1.93046689   2.99354314 109.53636472 109.58870908
 101.03116965]

[[-1.39039826 -1.36540555 -0.19557502]
 [ 1.38264787 -1.39015007 -0.20584254]
 [ 0.01228162  0.37514514  3.1007029 ]]
[  1.95851709   1.97144706   3.12333846 100.69566209 100.69979222
 